In [1]:
import argparse
import cv2
import glob
import os
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from gfpgan import GFPGANer
from realesrgan import RealESRGANer
from realesrgan.archs.srvgg_arch import SRVGGNetCompact
import tensorflow as tf
import time
from skimage import exposure
from numba import cuda
import torch
import gc


EXTERNAL = False

if EXTERNAL:
  %store -r WIDTH
  %store -r HEIGHT
  %store -r TEXT_IMAGE_GEN  
  %store -r SAVE_FOLDER
  %store -r PROJECT_NAME
  %store -r SHOW_OUTPUT
  %store -r IMG_NUMBER
  %store -r EXCLUDE_IMG_GEN
  %store -r PROMPT_IMG
  %store -r TEXT_VOICE_GEN
  %store -r NEGATIVE_P
  %store -r MODEL_14
  %store -r RECORD_VIDEO_AFTER
  %store -r FPS
else:
  RECORD_VIDEO_AFTER = 0
  WIDTH = 768
  HEIGHT = 768
  SAVE_FOLDER = "D:\\Deletar\\p_gen"
  PROJECT_NAME = "THE FELLOWSHIP OF THE RING"
  SHOW_OUTPUT = True
  IMG_NUMBER = 2
  EXCLUDE_IMG_GEN = []
  PROMPT_IMG = "((((beautiful))),: "
  TEXT_IMAGE_GEN = ['One punch man agains goku.', 'Vegeta against Freeza']
  TEXT_VOICE_GEN = ['a', "b"]
  MODEL_14 = False
  NEGATIVE_P = "ugly, morbid, cloned face, missing legs, tiling, extra fingers, mutation, bad anatomy,gross proportions, \
   missing arm, long neck, duplicate, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, \
   disfigured, deformed, body out of frame, blurry, blurred, watermark, grainy, signature, cut off, draft, "
  FPS = 15


ANIME = False

path = "./r.png"
save_path = "./modified.png"

netscale = 4
model_path_face = "./realesrgan/GFPGANv1.3.pth"
dni_weight = None

tile = 0
title_pad = 10
pre_pad = 0
half = None
gpu_id = None
upscale = 3.5
upscale = 4
img_mode = 'RGBA'

if ANIME:
  # R-ESRGAN + Anime
  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=4)
  model_path_x4 = "./realesrgan/RealESRGAN_x4plus_anime_6B.pth"
else:
  # R-ESRGAN
  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
  model_path_x4 = "./realesrgan/RealESRGAN_x4plus.pth"

project_folder = f"{SAVE_FOLDER}//{PROJECT_NAME}"

upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path_x4,
        dni_weight=dni_weight,
        model=model,
        tile=tile,
        tile_pad=title_pad,
        pre_pad=pre_pad,
        half= not half,
        gpu_id= gpu_id
)

face_enhancer = GFPGANer(
            model_path=model_path_face,
            upscale=upscale,
            arch='clean',
            channel_multiplier=2,
            bg_upsampler=upsampler)

def resize_image(data):
  resized_img = tf.image.resize(
  images=data,
  # size=[270, 480],     
  size=[512, 512], 
  method=tf.image.ResizeMethod.BILINEAR,
  preserve_aspect_ratio=False,
  antialias=True,
  )    
  return resized_img

def resize_image2(data):
  resized_img = tf.image.resize(
  images=data,
  size=[270, 480],
  method=tf.image.ResizeMethod.BILINEAR,
  preserve_aspect_ratio=False,
  antialias=True,
  )    
  return resized_img

def delete_files(path_to_del):
    folder_contents = os.listdir(path_to_del)
    for item in folder_contents:
        item_path = os.path.join(path_to_del, item)  # Get the full path to the item
        if os.path.isfile(item_path):
            os.remove(item_path)  # Delete the file
        elif os.path.isdir(item_path):
            os.rmdir(item_path)  # Delete the directory


c:\Users\Matheus\anaconda3\envs\Pyflow\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Matheus\anaconda3\envs\Pyflow\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
if EXTERNAL:
    if (not os.path.exists(f"{project_folder}\\img_hd")):
        os.makedirs(f"{project_folder}\\img_hd")
    for index, folder in enumerate(os.listdir(f"{project_folder}\\img")):
        if (not os.path.exists(f"{project_folder}\\img_hd\\{index:07d}")):
            os.makedirs(f"{project_folder}\\img_hd\\{index:07d}")
        delete_files(f"{project_folder}\\img_hd\\{index:07d}")
        cont = 0
        for file in os.listdir(f"{project_folder}\\img\\{folder}"):
            if(file[-3:] == 'png'):
                img = cv2.imread(f"{project_folder}\\img\\{folder}\\{file}", cv2.IMREAD_UNCHANGED)
                img = resize_image2(img).numpy()
                _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
                cv2.imwrite(f"{project_folder}\\img_hd\\{index:07d}\\{cont:05d}.png", output)
                cont += 1
                if (file[0] != 't'):
                    cv2.imwrite(f"{project_folder}\\img_hd\\{index:07d}\\{cont:05d}.png", output)
                    cont += 1
    """     cuda.select_device(0)
    cuda.close() """
    torch.cuda.empty_cache()
    gc.collect()

else:
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)

    img = resize_image(img).numpy()
    start = time.time() 
    _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
    # output = exposure.equalize_hist(output)*255
    # output, _ = upsampler.enhance(img, outscale=upscale)

    end = time.time()

    print(end-start)
    cv2.imwrite(save_path, output)

7.3964011669158936
